# Préparation des données pour l'article 
Ce bloc-notes décrit la préparation préliminaire des données effectuée pour mon article  sur la création d'un robot de diagnostic médical à l'aide de rasa.

La partie 1 décrit comment préparer les données d'entrée pour la partie diagnostic du projet tandis que la partie 2 décrit comment j'ai généré des données de training pour le modèle NLU de Rasa.

In [62]:
#pip install xlrd
import pandas as pd
import spacy
import random
import re

nlp = spacy.load('fr_core_news_md')




# Partie 1 : Préparation des données vectorielles

# Les fonctions
Ici, nous avons juste une fonction get_sentence_vector. Cette fonction prend simplement du texte et un objet NLP spacy et convertit le texte en une représentation vectorielle 246D.

In [63]:
def get_sentence_vectors(text, nlp):
    
    # get tokens for each word in sentence
    embedding = nlp(text).vector.tolist()
    
    # return mean token
    return embedding


# # Importer et fusionner des données

Nous ne nous soucions que de 3 fichiers téléchargés à partir de Kaggle `dia_t.csv` (liste des maladies avec identifiants), `sym_t.csv` (liste des symptômes avec identifiants) et `diffsydiw.csv` (mapping  des identifiants de maladie à divers symptômes identifiants).

une fois importé, nous pouvons tout réunir sur les colonnes d'identification et effectuer un nettoyage de base.

In [64]:
illness_df = pd.read_csv('medicaments.csv')
symptom_df = pd.read_csv('symptoms.csv')
links_df = pd.read_csv('ids.csv')


source_data = (links_df
               .merge(illness_df, on="mid")
               .merge(symptom_df, on="syd"))



In [65]:
source_data

,syd,mid,medicament,symptom
0,1,1,intoxication alimentaire,nausée
1,2,1,intoxication alimentaire,vomissement
2,3,1,intoxication alimentaire,diarrhée
3,4,1,intoxication alimentaire,fièvre
4,5,1,intoxication alimentaire,crampes d'estomac
5,6,2,Covid,fièvre
6,7,2,Covid,toux
7,8,2,Covid,fatigue
8,9,2,Covid,perte de l’odorat ou du goût


In [66]:
source_data = source_data.loc[~(source_data['symptom'].isna())
                             & ~(source_data['medicament'].isna()),
                             ['mid', 'syd', 'medicament', 'symptom']]
source_data.columns = ['illness_id', 'symptom_id', 'illness', 'symptom']

source_data['illness'] = source_data['illness'].str.replace('\x0b', ' ')
source_data['symptom'] = source_data['symptom'].str.replace('\x0b', ' ')



# ## Utilisez notre fonction pour convertir chaque symptôme en une représentation vectorielle, nous pouvons ensuite enregistrer nos données de symptômes dans le dossier de données

In [67]:
symptom_df = symptom_df.loc[~symptom_df['symptom'].isna()]
symptom_df['embedding'] = symptom_df.apply(lambda row: get_sentence_vectors(row['symptom'], nlp), axis = 1)
symptom_df.columns = ['syd', 'symptom','symptom_vector']
#symptom_df = symptom_df.loc[~symptom_df['symptom'].isna()]
# remove any messy characters
symptom_df['symptom'] = symptom_df['symptom'].str.replace('\x0b', ' ')



In [68]:


source_data.to_pickle('source_data.pkl')
symptom_df.to_pickle('symptoms.pkl')

In [69]:
symptom_df

,syd,symptom,symptom_vector
0,1,nausée,"[1.9656000137329102, -1.8554999828338623, -1.9..."
1,2,vomissement,"[1.0999000072479248, -1.440500020980835, -1.77..."
2,3,diarrhée,"[3.8129000663757324, 0.3746199905872345, -3.78..."
3,4,fièvre,"[3.3090999126434326, -0.4234899878501892, -0.9..."
4,5,crampes d'estomac,"[1.5947399139404297, -1.2060266733169556, -2.9..."
5,6,fièvre,"[3.3090999126434326, -0.4234899878501892, -0.9..."
6,7,toux,"[3.8129000663757324, 0.3746199905872345, -3.78..."
7,8,fatigue,"[2.547499895095825, 0.10085999965667725, -0.49..."
8,9,perte de l’odorat ou du goût,"[0.010998555459082127, -2.766038656234741, 0.5..."


# Créer un vecteur de symptômes pour chaque maladie décrite dans les données
Ici, nous allons parcourir chaque maladie décrite dans l'ensemble de données et marquer (avec un 1 ou 0) chacun des symptômes de cette maladie. Nous comparerons ensuite une liste de symptômes signalés décrits par l'utilisateur à celle-ci en utilisant une fonction de similarité cosinus pour diagnostiquer une maladie potentielle.

In [70]:
# list of illness
illnesses = list(source_data['illness'].drop_duplicates())

# list we will use to store our illness vectors
symptom_vectors = []

for illness in illnesses:
    
    illness_symptoms = list(source_data.loc[source_data["illness"]==illness, 'symptom'].drop_duplicates())
    
    symptom_df["related_to_illness"] = 0
    symptom_df.loc[symptom_df["symptom"].isin(illness_symptoms), "related_to_illness"] = 1
    
    
    symptom_vectors.append(list(symptom_df["related_to_illness"]))
    
diagnosis_data = pd.DataFrame({"illness":illnesses,
                              "illness_vector": symptom_vectors})


diagnosis_data.to_pickle('diagnosis_data.pkl')


In [71]:
diagnosis_data

,illness,illness_vector
0,intoxication alimentaire,"[1, 1, 1, 1, 1, 1, 0, 0, 0]"
1,Covid,"[0, 0, 0, 1, 0, 1, 1, 1, 1]"


# Partie 2 : Générer des échantillons d'apprentissage avec des entités signalées¶
Dans cette partie, nous tenterons de générer des exemples de formation pour le modèle NLU afin de comprendre comment nos utilisateurs décriront les symptômes au chat bot. Pour ce faire, nous parcourons simplement nos symptômes, les combinant parfois avant de les ajouter à différents débuts/fins.

Nous nous assurons également de marquer toutes les descriptions de symptômes en tant qu'entités de symptômes, qui seront comprises par le modèle NLU de Rasa. Vous pouvez voir que cela est accompli en insérant '(symptôme)' à côté de toute mention de symptômes.

La sortie de ceci est insérée dans nlu.md dans le projet principal

In [72]:



number_of_symptoms = [1, 2, 3, 4]
start_of_description = [
    "J'ai",
     "Je souffre de",
     "J'ai vraiment mal",
     "Mes symptômes sont",
     "Depuis quelques jours, j'ai eu",
     "Mon mari souffre de",
     "Ma femme souffre de",
     "Mon fils souffre de",
     "Ma fille souffre de",
     "Mon enfant souffre de",
     "Je ne me sens pas bien, j'ai"
]

# get some examples of users describing different numbers of syptoms
for symptons_count in number_of_symptoms:
    
    # make 100 examples of each number of symptoms
    for ex in range(1, 101):
    
        description_beginning = random.choice(start_of_description)
        
        # collect some random symtpoms
        symptom_1 = symptom_df['symptom'].sample(1).iloc[0].lower()
        symptom_2 = symptom_df['symptom'].sample(1).iloc[0].lower()
        symptom_3 = symptom_df['symptom'].sample(1).iloc[0].lower()
        symptom_4 = symptom_df['symptom'].sample(1).iloc[0].lower()
        
        symptoms = [symptom_1, symptom_2, symptom_3, symptom_4]
        symptoms_entity = []
        
        # remove parenthases from symptoms and add nessecary entitiy tags to symptoms
        for symptom in symptoms:
            symptom = re.sub(r"\([^)]+\)", "", symptom).strip()
            symptom = f"[{symptom}](symptom)"
            symptoms_entity.append(symptom)
            
        symptom_1 = symptoms_entity[0]
        symptom_2 = symptoms_entity[1]
        symptom_3 = symptoms_entity[2]
        symptom_4 = symptoms_entity[3]
        
        # create the training sample string
        if symptons_count == 1:
            
            symptom_string = f"- {description_beginning} {symptom_1}"
            
        if symptons_count == 2:
            
            symptom_string = f"- {description_beginning} {symptom_1} and {symptom_2}"
            
        if symptons_count == 3:
            
            symptom_string = f"- {description_beginning} {symptom_1}, {symptom_2}, and {symptom_3}"
            
        if symptons_count == 4:
            
            symptom_string = f"- {description_beginning} {symptom_1}, {symptom_2}, {symptom_3}, {symptom_4}"
        
        print(symptom_string)

- Ma femme souffre de [toux](symptom)
- Depuis quelques jours, j'ai eu [crampes d'estomac](symptom)
- Mon fils souffre de [toux](symptom)
- Depuis quelques jours, j'ai eu [crampes d'estomac](symptom)
- Ma femme souffre de [diarrhée](symptom)
- Mon fils souffre de [vomissement](symptom)
- Depuis quelques jours, j'ai eu [crampes d'estomac](symptom)
- Mes symptômes sont [crampes d'estomac](symptom)
- Mon enfant souffre de [toux](symptom)
- J'ai [fatigue](symptom)
- J'ai vraiment mal [nausée](symptom)
- Je souffre de [fièvre](symptom)
- Ma femme souffre de [toux](symptom)
- Je ne me sens pas bien, j'ai [fièvre](symptom)
- Depuis quelques jours, j'ai eu [fièvre](symptom)
- Ma femme souffre de [vomissement](symptom)
- Mes symptômes sont [toux](symptom)
- Mes symptômes sont [crampes d'estomac](symptom)
- J'ai [toux](symptom)
- Ma femme souffre de [diarrhée](symptom)
- J'ai vraiment mal [fièvre](symptom)
- Je ne me sens pas bien, j'ai [nausée](symptom)
- Mon enfant souffre de [fièvre](symptom)

- Je souffre de [fièvre](symptom), [nausée](symptom), [nausée](symptom), [fièvre](symptom)
- J'ai vraiment mal [fièvre](symptom), [fièvre](symptom), [fatigue](symptom), [fatigue](symptom)
- Ma femme souffre de [nausée](symptom), [diarrhée](symptom), [perte de l’odorat ou du goût](symptom), [toux](symptom)
- Depuis quelques jours, j'ai eu [fièvre](symptom), [diarrhée](symptom), [toux](symptom), [perte de l’odorat ou du goût](symptom)
- Depuis quelques jours, j'ai eu [vomissement](symptom), [toux](symptom), [diarrhée](symptom), [perte de l’odorat ou du goût](symptom)
- Je ne me sens pas bien, j'ai [nausée](symptom), [vomissement](symptom), [toux](symptom), [vomissement](symptom)
- Mon fils souffre de [fatigue](symptom), [nausée](symptom), [crampes d'estomac](symptom), [fatigue](symptom)
- Depuis quelques jours, j'ai eu [perte de l’odorat ou du goût](symptom), [fièvre](symptom), [diarrhée](symptom), [diarrhée](symptom)
- Mon mari souffre de [diarrhée](symptom), [vomissement](symptom), [fi

In [73]:
- Mon mari souffre de [fatigue](symptom)
- Ma fille souffre de [angines](symptom)
- Ma femme souffre de [diarrhée](symptom)
- Ma femme souffre de [champignon](symptom)
- Mes symptômes sont [maux de tete](symptom)
- Je souffre de [fatigue](symptom)
- Mes symptômes sont [diarrhée](symptom)
- Je ne me sens pas bien, j'ai [champignon](symptom)
- J'ai [maux de tete](symptom)
- Mon mari souffre de [fatigue](symptom)
- Mon mari souffre de [fatigue](symptom)
- Mon mari souffre de [fatigue](symptom)
- J'ai [maux de tete](symptom)
- Mon fils souffre de [champignon](symptom)
- Mon mari souffre de [maux de tete](symptom)
- Depuis quelques jours, j'ai eu [angines](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom)
- Je souffre de [maux de tete](symptom)
- Je ne me sens pas bien, j'ai [maux de tete](symptom)
- Ma femme souffre de [maux de tete](symptom)
- Ma femme souffre de [champignon](symptom)
- J'ai vraiment mal [fatigue](symptom)
- Je souffre de [angines](symptom)
- Mon mari souffre de [champignon](symptom)
- J'ai [champignon](symptom)
- Depuis quelques jours, j'ai eu [angines](symptom)
- Ma fille souffre de [diarrhée](symptom)
- Mon fils souffre de [angines](symptom)
- J'ai [angines](symptom)
- Mon enfant souffre de [fatigue](symptom)
- J'ai vraiment mal [angines](symptom)
- Je souffre de [maux de tete](symptom)
- Mes symptômes sont [champignon](symptom)
- Mon mari souffre de [fatigue](symptom)
- Je ne me sens pas bien, j'ai [maux de tete](symptom)
- J'ai [champignon](symptom)
- Je souffre de [champignon](symptom)
- Je souffre de [angines](symptom)
- J'ai [angines](symptom)
- Je souffre de [diarrhée](symptom)
- Mon enfant souffre de [maux de tete](symptom)
- Mon enfant souffre de [fatigue](symptom)
- Je souffre de [angines](symptom)
- J'ai [angines](symptom)
- Mon mari souffre de [maux de tete](symptom)
- J'ai [angines](symptom)
- Mes symptômes sont [angines](symptom)
- Mon fils souffre de [champignon](symptom)
- Je souffre de [fatigue](symptom)
- Je souffre de [maux de tete](symptom)
- Je souffre de [diarrhée](symptom)
- J'ai vraiment mal [champignon](symptom)
- J'ai [maux de tete](symptom)
- Mon enfant souffre de [fatigue](symptom)
- J'ai vraiment mal [fatigue](symptom)
- Depuis quelques jours, j'ai eu [champignon](symptom)
- J'ai vraiment mal [angines](symptom)
- Ma fille souffre de [champignon](symptom)
- Ma fille souffre de [champignon](symptom)
- Ma fille souffre de [champignon](symptom)
- Mes symptômes sont [champignon](symptom)
- Depuis quelques jours, j'ai eu [angines](symptom)
- Je souffre de [diarrhée](symptom)
- Je ne me sens pas bien, j'ai [maux de tete](symptom)
- Ma femme souffre de [maux de tete](symptom)
- J'ai vraiment mal [fatigue](symptom)
- Mes symptômes sont [champignon](symptom)
- Mon enfant souffre de [fatigue](symptom)
- Mon enfant souffre de [champignon](symptom)
- J'ai vraiment mal [angines](symptom)
- Mes symptômes sont [fatigue](symptom)
- Mes symptômes sont [maux de tete](symptom)
- Je souffre de [angines](symptom)
- Je ne me sens pas bien, j'ai [champignon](symptom)
- Mon fils souffre de [fatigue](symptom)
- J'ai [maux de tete](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom)
- Depuis quelques jours, j'ai eu [champignon](symptom)
- Mes symptômes sont [fatigue](symptom)
- J'ai vraiment mal [maux de tete](symptom)
- Ma fille souffre de [angines](symptom)
- Depuis quelques jours, j'ai eu [angines](symptom)
- Ma femme souffre de [fatigue](symptom)
- Mon enfant souffre de [fatigue](symptom)
- Ma fille souffre de [champignon](symptom)
- J'ai [diarrhée](symptom)
- J'ai [maux de tete](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom)
- Mes symptômes sont [diarrhée](symptom)
- Je ne me sens pas bien, j'ai [angines](symptom)
- J'ai vraiment mal [maux de tete](symptom)
- Depuis quelques jours, j'ai eu [angines](symptom)
- J'ai vraiment mal [champignon](symptom)
- J'ai [maux de tete](symptom)
- J'ai [diarrhée](symptom)
- Mes symptômes sont [maux de tete](symptom)
- Ma fille souffre de [fatigue](symptom)
- Mon mari souffre de [diarrhée](symptom)
- Mes symptômes sont [champignon](symptom)
- Je souffre de [champignon](symptom)
- Mon enfant souffre de [fatigue](symptom) and [diarrhée](symptom)
- Mon mari souffre de [champignon](symptom) and [angines](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom) and [diarrhée](symptom)
- Mon fils souffre de [champignon](symptom) and [champignon](symptom)
- Depuis quelques jours, j'ai eu [diarrhée](symptom) and [maux de tete](symptom)
- Ma fille souffre de [champignon](symptom) and [fatigue](symptom)
- Mes symptômes sont [champignon](symptom) and [fatigue](symptom)
- Mon enfant souffre de [champignon](symptom) and [champignon](symptom)
- Mon fils souffre de [diarrhée](symptom) and [diarrhée](symptom)
- J'ai vraiment mal [angines](symptom) and [maux de tete](symptom)
- Je ne me sens pas bien, j'ai [diarrhée](symptom) and [diarrhée](symptom)
- Mon enfant souffre de [champignon](symptom) and [fatigue](symptom)
- J'ai [champignon](symptom) and [angines](symptom)
- Mes symptômes sont [diarrhée](symptom) and [diarrhée](symptom)
- Mon mari souffre de [maux de tete](symptom) and [angines](symptom)
- Depuis quelques jours, j'ai eu [diarrhée](symptom) and [maux de tete](symptom)
- Depuis quelques jours, j'ai eu [diarrhée](symptom) and [champignon](symptom)
- J'ai [angines](symptom) and [angines](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom) and [diarrhée](symptom)
- J'ai vraiment mal [maux de tete](symptom) and [maux de tete](symptom)
- Ma femme souffre de [fatigue](symptom) and [champignon](symptom)
- Mon fils souffre de [fatigue](symptom) and [maux de tete](symptom)
- Mon enfant souffre de [diarrhée](symptom) and [diarrhée](symptom)
- Ma fille souffre de [angines](symptom) and [champignon](symptom)
- Mes symptômes sont [diarrhée](symptom) and [champignon](symptom)
- Ma fille souffre de [champignon](symptom) and [diarrhée](symptom)
- Mon fils souffre de [angines](symptom) and [maux de tete](symptom)
- J'ai [maux de tete](symptom) and [maux de tete](symptom)
- Mon fils souffre de [angines](symptom) and [angines](symptom)
- J'ai vraiment mal [champignon](symptom) and [angines](symptom)
- Je souffre de [fatigue](symptom) and [angines](symptom)
- Ma femme souffre de [diarrhée](symptom) and [fatigue](symptom)
- Mon mari souffre de [diarrhée](symptom) and [champignon](symptom)
- Je ne me sens pas bien, j'ai [diarrhée](symptom) and [angines](symptom)
- J'ai vraiment mal [champignon](symptom) and [angines](symptom)
- Mes symptômes sont [maux de tete](symptom) and [fatigue](symptom)
- Ma femme souffre de [diarrhée](symptom) and [champignon](symptom)
- J'ai vraiment mal [maux de tete](symptom) and [maux de tete](symptom)
- Mes symptômes sont [fatigue](symptom) and [champignon](symptom)
- Mes symptômes sont [champignon](symptom) and [fatigue](symptom)
- Mes symptômes sont [maux de tete](symptom) and [fatigue](symptom)
- Je souffre de [fatigue](symptom) and [angines](symptom)
- J'ai vraiment mal [maux de tete](symptom) and [champignon](symptom)
- Ma femme souffre de [angines](symptom) and [maux de tete](symptom)
- Je souffre de [diarrhée](symptom) and [fatigue](symptom)
- Mon enfant souffre de [diarrhée](symptom) and [angines](symptom)
- Depuis quelques jours, j'ai eu [diarrhée](symptom) and [champignon](symptom)
- Mon enfant souffre de [angines](symptom) and [angines](symptom)
- Ma fille souffre de [fatigue](symptom) and [fatigue](symptom)
- Mon enfant souffre de [fatigue](symptom) and [fatigue](symptom)
- Mes symptômes sont [fatigue](symptom) and [angines](symptom)
- Depuis quelques jours, j'ai eu [fatigue](symptom) and [diarrhée](symptom)
- Depuis quelques jours, j'ai eu [diarrhée](symptom) and [diarrhée](symptom)
- Ma femme souffre de [maux de tete](symptom) and [champignon](symptom)
- Ma fille souffre de [fatigue](symptom) and [angines](symptom)
- Je souffre de [diarrhée](symptom) and [angines](symptom)
- Mes symptômes sont [fatigue](symptom) and [fatigue](symptom)
- Je ne me sens pas bien, j'ai [angines](symptom) and [fatigue](symptom)
- Mon mari souffre de [maux de tete](symptom) and [maux de tete](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom) and [maux de tete](symptom)
- Ma fille souffre de [angines](symptom) and [maux de tete](symptom)
- Ma femme souffre de [diarrhée](symptom) and [angines](symptom)
- Mon mari souffre de [diarrhée](symptom) and [angines](symptom)
- Mes symptômes sont [maux de tete](symptom) and [fatigue](symptom)
- Mes symptômes sont [fatigue](symptom) and [diarrhée](symptom)
- J'ai vraiment mal [diarrhée](symptom) and [maux de tete](symptom)
- J'ai vraiment mal [maux de tete](symptom) and [fatigue](symptom)
- J'ai [diarrhée](symptom) and [diarrhée](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom) and [fatigue](symptom)
- Je souffre de [champignon](symptom) and [diarrhée](symptom)
- Mes symptômes sont [fatigue](symptom) and [fatigue](symptom)
- J'ai [fatigue](symptom) and [champignon](symptom)
- J'ai [fatigue](symptom) and [diarrhée](symptom)
- Depuis quelques jours, j'ai eu [angines](symptom) and [angines](symptom)
- Ma fille souffre de [champignon](symptom) and [champignon](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom) and [angines](symptom)
- Mon fils souffre de [diarrhée](symptom) and [champignon](symptom)
- Ma fille souffre de [maux de tete](symptom) and [champignon](symptom)
- Mon fils souffre de [champignon](symptom) and [maux de tete](symptom)
- Ma femme souffre de [angines](symptom) and [fatigue](symptom)
- Mes symptômes sont [fatigue](symptom) and [angines](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom) and [fatigue](symptom)
- Mon mari souffre de [maux de tete](symptom) and [angines](symptom)
- Mon fils souffre de [fatigue](symptom) and [angines](symptom)
- Ma fille souffre de [maux de tete](symptom) and [maux de tete](symptom)
- Ma femme souffre de [diarrhée](symptom) and [fatigue](symptom)
- Mon fils souffre de [champignon](symptom) and [maux de tete](symptom)
- Je ne me sens pas bien, j'ai [diarrhée](symptom) and [maux de tete](symptom)
- Mes symptômes sont [champignon](symptom) and [champignon](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom) and [champignon](symptom)
- Mon fils souffre de [maux de tete](symptom) and [diarrhée](symptom)
- Mon fils souffre de [champignon](symptom) and [diarrhée](symptom)
- J'ai vraiment mal [fatigue](symptom) and [angines](symptom)
- Mon enfant souffre de [maux de tete](symptom) and [angines](symptom)
- Mon fils souffre de [fatigue](symptom) and [fatigue](symptom)
- Mon fils souffre de [angines](symptom) and [maux de tete](symptom)
- J'ai vraiment mal [champignon](symptom) and [fatigue](symptom)
- Depuis quelques jours, j'ai eu [diarrhée](symptom) and [maux de tete](symptom)
- Mon fils souffre de [champignon](symptom) and [diarrhée](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom) and [angines](symptom)
- Ma fille souffre de [fatigue](symptom), [diarrhée](symptom), and [diarrhée](symptom)
- Je souffre de [fatigue](symptom), [maux de tete](symptom), and [champignon](symptom)
- Je souffre de [angines](symptom), [diarrhée](symptom), and [maux de tete](symptom)
- Ma fille souffre de [angines](symptom), [champignon](symptom), and [diarrhée](symptom)
- Ma fille souffre de [maux de tete](symptom), [angines](symptom), and [diarrhée](symptom)
- J'ai vraiment mal [champignon](symptom), [maux de tete](symptom), and [fatigue](symptom)
- Mes symptômes sont [fatigue](symptom), [champignon](symptom), and [diarrhée](symptom)
- J'ai [diarrhée](symptom), [diarrhée](symptom), and [angines](symptom)
- Mon enfant souffre de [angines](symptom), [maux de tete](symptom), and [maux de tete](symptom)
- Mon fils souffre de [champignon](symptom), [maux de tete](symptom), and [fatigue](symptom)
- Mon enfant souffre de [maux de tete](symptom), [angines](symptom), and [diarrhée](symptom)
- Depuis quelques jours, j'ai eu [champignon](symptom), [maux de tete](symptom), and [champignon](symptom)
- Depuis quelques jours, j'ai eu [diarrhée](symptom), [maux de tete](symptom), and [diarrhée](symptom)
- Je ne me sens pas bien, j'ai [angines](symptom), [maux de tete](symptom), and [fatigue](symptom)
- Mes symptômes sont [champignon](symptom), [champignon](symptom), and [diarrhée](symptom)
- Mon enfant souffre de [champignon](symptom), [maux de tete](symptom), and [diarrhée](symptom)
- Mes symptômes sont [champignon](symptom), [maux de tete](symptom), and [fatigue](symptom)
- Mon enfant souffre de [champignon](symptom), [angines](symptom), and [maux de tete](symptom)
- Ma femme souffre de [maux de tete](symptom), [fatigue](symptom), and [angines](symptom)
- Mon enfant souffre de [champignon](symptom), [fatigue](symptom), and [maux de tete](symptom)
- Je ne me sens pas bien, j'ai [diarrhée](symptom), [diarrhée](symptom), and [fatigue](symptom)
- Je souffre de [diarrhée](symptom), [fatigue](symptom), and [champignon](symptom)
- J'ai vraiment mal [diarrhée](symptom), [champignon](symptom), and [maux de tete](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom), [angines](symptom), and [champignon](symptom)
- Mon mari souffre de [maux de tete](symptom), [diarrhée](symptom), and [maux de tete](symptom)
- J'ai [fatigue](symptom), [angines](symptom), and [diarrhée](symptom)
- Mon fils souffre de [maux de tete](symptom), [maux de tete](symptom), and [diarrhée](symptom)
- Mon mari souffre de [angines](symptom), [champignon](symptom), and [diarrhée](symptom)
- Je ne me sens pas bien, j'ai [angines](symptom), [champignon](symptom), and [diarrhée](symptom)
- Mon enfant souffre de [champignon](symptom), [fatigue](symptom), and [champignon](symptom)
- Mon fils souffre de [angines](symptom), [maux de tete](symptom), and [angines](symptom)
- Je souffre de [fatigue](symptom), [champignon](symptom), and [champignon](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom), [angines](symptom), and [fatigue](symptom)
- J'ai vraiment mal [fatigue](symptom), [angines](symptom), and [angines](symptom)
- Mon enfant souffre de [fatigue](symptom), [diarrhée](symptom), and [angines](symptom)
- Mon mari souffre de [champignon](symptom), [fatigue](symptom), and [angines](symptom)
- Mes symptômes sont [champignon](symptom), [champignon](symptom), and [diarrhée](symptom)
- Je ne me sens pas bien, j'ai [maux de tete](symptom), [diarrhée](symptom), and [angines](symptom)
- J'ai [diarrhée](symptom), [fatigue](symptom), and [angines](symptom)
- J'ai vraiment mal [champignon](symptom), [diarrhée](symptom), and [champignon](symptom)
- Mon fils souffre de [diarrhée](symptom), [fatigue](symptom), and [diarrhée](symptom)
- J'ai [fatigue](symptom), [angines](symptom), and [diarrhée](symptom)
- Mes symptômes sont [fatigue](symptom), [fatigue](symptom), and [angines](symptom)
- Ma fille souffre de [maux de tete](symptom), [angines](symptom), and [maux de tete](symptom)
- Ma femme souffre de [champignon](symptom), [maux de tete](symptom), and [maux de tete](symptom)
- J'ai vraiment mal [diarrhée](symptom), [champignon](symptom), and [diarrhée](symptom)
- Depuis quelques jours, j'ai eu [champignon](symptom), [diarrhée](symptom), and [diarrhée](symptom)
- Depuis quelques jours, j'ai eu [angines](symptom), [fatigue](symptom), and [fatigue](symptom)
- Mes symptômes sont [champignon](symptom), [fatigue](symptom), and [maux de tete](symptom)
- Mon fils souffre de [angines](symptom), [angines](symptom), and [maux de tete](symptom)
- J'ai [champignon](symptom), [fatigue](symptom), and [maux de tete](symptom)
- Ma fille souffre de [champignon](symptom), [fatigue](symptom), and [champignon](symptom)
- J'ai [angines](symptom), [maux de tete](symptom), and [maux de tete](symptom)
- Mes symptômes sont [angines](symptom), [diarrhée](symptom), and [champignon](symptom)
- Depuis quelques jours, j'ai eu [angines](symptom), [fatigue](symptom), and [angines](symptom)
- Depuis quelques jours, j'ai eu [fatigue](symptom), [fatigue](symptom), and [fatigue](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom), [diarrhée](symptom), and [fatigue](symptom)
- Mon mari souffre de [angines](symptom), [fatigue](symptom), and [champignon](symptom)
- Ma fille souffre de [diarrhée](symptom), [maux de tete](symptom), and [champignon](symptom)
- J'ai vraiment mal [diarrhée](symptom), [angines](symptom), and [champignon](symptom)
- Mon fils souffre de [diarrhée](symptom), [fatigue](symptom), and [angines](symptom)
- Ma femme souffre de [champignon](symptom), [champignon](symptom), and [diarrhée](symptom)
- Mon fils souffre de [angines](symptom), [champignon](symptom), and [maux de tete](symptom)
- Mes symptômes sont [fatigue](symptom), [fatigue](symptom), and [angines](symptom)
- Depuis quelques jours, j'ai eu [fatigue](symptom), [maux de tete](symptom), and [angines](symptom)
- Ma fille souffre de [maux de tete](symptom), [diarrhée](symptom), and [champignon](symptom)
- Je souffre de [maux de tete](symptom), [angines](symptom), and [fatigue](symptom)
- Mon fils souffre de [champignon](symptom), [maux de tete](symptom), and [angines](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom), [champignon](symptom), and [fatigue](symptom)
- Depuis quelques jours, j'ai eu [angines](symptom), [fatigue](symptom), and [fatigue](symptom)
- J'ai vraiment mal [diarrhée](symptom), [champignon](symptom), and [diarrhée](symptom)
- J'ai [champignon](symptom), [champignon](symptom), and [maux de tete](symptom)
- Ma femme souffre de [maux de tete](symptom), [maux de tete](symptom), and [diarrhée](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom), [angines](symptom), and [maux de tete](symptom)
- Mon mari souffre de [maux de tete](symptom), [angines](symptom), and [angines](symptom)
- Mon fils souffre de [champignon](symptom), [maux de tete](symptom), and [champignon](symptom)
- Je souffre de [maux de tete](symptom), [champignon](symptom), and [champignon](symptom)
- J'ai vraiment mal [fatigue](symptom), [fatigue](symptom), and [maux de tete](symptom)
- Mes symptômes sont [diarrhée](symptom), [angines](symptom), and [diarrhée](symptom)
- Mes symptômes sont [champignon](symptom), [champignon](symptom), and [maux de tete](symptom)
- Mes symptômes sont [fatigue](symptom), [diarrhée](symptom), and [maux de tete](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom), [maux de tete](symptom), and [angines](symptom)
- Mon mari souffre de [angines](symptom), [champignon](symptom), and [fatigue](symptom)
- J'ai [fatigue](symptom), [diarrhée](symptom), and [angines](symptom)
- Je souffre de [maux de tete](symptom), [maux de tete](symptom), and [fatigue](symptom)
- Mon enfant souffre de [maux de tete](symptom), [fatigue](symptom), and [angines](symptom)
- Ma femme souffre de [fatigue](symptom), [maux de tete](symptom), and [diarrhée](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom), [maux de tete](symptom), and [maux de tete](symptom)
- J'ai [champignon](symptom), [champignon](symptom), and [diarrhée](symptom)
- Ma femme souffre de [angines](symptom), [fatigue](symptom), and [champignon](symptom)
- Depuis quelques jours, j'ai eu [fatigue](symptom), [diarrhée](symptom), and [fatigue](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom), [angines](symptom), and [fatigue](symptom)
- Mon fils souffre de [angines](symptom), [champignon](symptom), and [diarrhée](symptom)
- J'ai vraiment mal [maux de tete](symptom), [diarrhée](symptom), and [diarrhée](symptom)
- Mes symptômes sont [fatigue](symptom), [champignon](symptom), and [angines](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom), [diarrhée](symptom), and [diarrhée](symptom)
- Ma femme souffre de [champignon](symptom), [diarrhée](symptom), and [diarrhée](symptom)
- Mon fils souffre de [maux de tete](symptom), [fatigue](symptom), and [angines](symptom)
- Ma femme souffre de [fatigue](symptom), [fatigue](symptom), and [champignon](symptom)
- Mon mari souffre de [maux de tete](symptom), [maux de tete](symptom), and [angines](symptom)
- J'ai vraiment mal [champignon](symptom), [champignon](symptom), [fatigue](symptom), [fatigue](symptom)
- Je souffre de [angines](symptom), [champignon](symptom), [fatigue](symptom), [champignon](symptom)
- Ma fille souffre de [champignon](symptom), [maux de tete](symptom), [maux de tete](symptom), [diarrhée](symptom)
- Ma fille souffre de [fatigue](symptom), [fatigue](symptom), [maux de tete](symptom), [maux de tete](symptom)
- Ma fille souffre de [champignon](symptom), [maux de tete](symptom), [maux de tete](symptom), [diarrhée](symptom)
- Je ne me sens pas bien, j'ai [angines](symptom), [diarrhée](symptom), [angines](symptom), [angines](symptom)
- Je ne me sens pas bien, j'ai [champignon](symptom), [fatigue](symptom), [fatigue](symptom), [maux de tete](symptom)
- Je ne me sens pas bien, j'ai [champignon](symptom), [angines](symptom), [diarrhée](symptom), [diarrhée](symptom)
- Je ne me sens pas bien, j'ai [champignon](symptom), [maux de tete](symptom), [maux de tete](symptom), [angines](symptom)
- Je souffre de [fatigue](symptom), [diarrhée](symptom), [maux de tete](symptom), [angines](symptom)
- Ma fille souffre de [diarrhée](symptom), [angines](symptom), [fatigue](symptom), [fatigue](symptom)
- Mon enfant souffre de [fatigue](symptom), [maux de tete](symptom), [maux de tete](symptom), [maux de tete](symptom)
- Mes symptômes sont [champignon](symptom), [fatigue](symptom), [champignon](symptom), [angines](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom), [maux de tete](symptom), [fatigue](symptom), [maux de tete](symptom)
- Mon mari souffre de [champignon](symptom), [diarrhée](symptom), [angines](symptom), [fatigue](symptom)
- Mon mari souffre de [maux de tete](symptom), [champignon](symptom), [champignon](symptom), [angines](symptom)
- J'ai [maux de tete](symptom), [angines](symptom), [diarrhée](symptom), [maux de tete](symptom)
- Mon mari souffre de [fatigue](symptom), [diarrhée](symptom), [diarrhée](symptom), [champignon](symptom)
- Je ne me sens pas bien, j'ai [angines](symptom), [fatigue](symptom), [champignon](symptom), [champignon](symptom)
- Je ne me sens pas bien, j'ai [angines](symptom), [angines](symptom), [fatigue](symptom), [angines](symptom)
- Mes symptômes sont [fatigue](symptom), [fatigue](symptom), [maux de tete](symptom), [fatigue](symptom)
- Ma femme souffre de [maux de tete](symptom), [fatigue](symptom), [maux de tete](symptom), [maux de tete](symptom)
- Mes symptômes sont [maux de tete](symptom), [champignon](symptom), [champignon](symptom), [champignon](symptom)
- J'ai vraiment mal [fatigue](symptom), [angines](symptom), [maux de tete](symptom), [diarrhée](symptom)
- Ma femme souffre de [diarrhée](symptom), [diarrhée](symptom), [fatigue](symptom), [fatigue](symptom)
- Je souffre de [maux de tete](symptom), [diarrhée](symptom), [champignon](symptom), [maux de tete](symptom)
- Mon enfant souffre de [diarrhée](symptom), [maux de tete](symptom), [maux de tete](symptom), [maux de tete](symptom)
- Mes symptômes sont [maux de tete](symptom), [maux de tete](symptom), [diarrhée](symptom), [champignon](symptom)
- Je souffre de [champignon](symptom), [maux de tete](symptom), [diarrhée](symptom), [champignon](symptom)
- Mon fils souffre de [champignon](symptom), [champignon](symptom), [maux de tete](symptom), [angines](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom), [maux de tete](symptom), [angines](symptom), [diarrhée](symptom)
- Mon enfant souffre de [champignon](symptom), [angines](symptom), [maux de tete](symptom), [maux de tete](symptom)
- Ma fille souffre de [maux de tete](symptom), [maux de tete](symptom), [fatigue](symptom), [diarrhée](symptom)
- Mon enfant souffre de [fatigue](symptom), [diarrhée](symptom), [maux de tete](symptom), [maux de tete](symptom)
- Mon fils souffre de [diarrhée](symptom), [maux de tete](symptom), [fatigue](symptom), [fatigue](symptom)
- Je souffre de [diarrhée](symptom), [maux de tete](symptom), [diarrhée](symptom), [maux de tete](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom), [champignon](symptom), [diarrhée](symptom), [fatigue](symptom)
- Mon mari souffre de [angines](symptom), [angines](symptom), [diarrhée](symptom), [fatigue](symptom)
- Mon fils souffre de [champignon](symptom), [maux de tete](symptom), [maux de tete](symptom), [champignon](symptom)
- Mes symptômes sont [maux de tete](symptom), [fatigue](symptom), [angines](symptom), [angines](symptom)
- Mes symptômes sont [fatigue](symptom), [angines](symptom), [champignon](symptom), [champignon](symptom)
- J'ai [champignon](symptom), [fatigue](symptom), [maux de tete](symptom), [fatigue](symptom)
- Mes symptômes sont [maux de tete](symptom), [fatigue](symptom), [diarrhée](symptom), [diarrhée](symptom)
- J'ai [angines](symptom), [champignon](symptom), [diarrhée](symptom), [maux de tete](symptom)
- Ma femme souffre de [diarrhée](symptom), [angines](symptom), [angines](symptom), [champignon](symptom)
- Mes symptômes sont [fatigue](symptom), [fatigue](symptom), [fatigue](symptom), [champignon](symptom)
- Mon mari souffre de [maux de tete](symptom), [maux de tete](symptom), [champignon](symptom), [champignon](symptom)
- Ma fille souffre de [fatigue](symptom), [champignon](symptom), [diarrhée](symptom), [champignon](symptom)
- J'ai [angines](symptom), [fatigue](symptom), [angines](symptom), [champignon](symptom)
- Mon enfant souffre de [champignon](symptom), [diarrhée](symptom), [angines](symptom), [maux de tete](symptom)
- Mon fils souffre de [maux de tete](symptom), [fatigue](symptom), [diarrhée](symptom), [fatigue](symptom)
- Ma fille souffre de [champignon](symptom), [maux de tete](symptom), [champignon](symptom), [fatigue](symptom)
- Je ne me sens pas bien, j'ai [diarrhée](symptom), [champignon](symptom), [maux de tete](symptom), [champignon](symptom)
- Mon mari souffre de [diarrhée](symptom), [angines](symptom), [maux de tete](symptom), [fatigue](symptom)
- Mon mari souffre de [fatigue](symptom), [angines](symptom), [fatigue](symptom), [maux de tete](symptom)
- Depuis quelques jours, j'ai eu [angines](symptom), [maux de tete](symptom), [champignon](symptom), [diarrhée](symptom)
- Mes symptômes sont [diarrhée](symptom), [angines](symptom), [champignon](symptom), [diarrhée](symptom)
- Ma fille souffre de [angines](symptom), [diarrhée](symptom), [fatigue](symptom), [champignon](symptom)
- Mon fils souffre de [angines](symptom), [angines](symptom), [maux de tete](symptom), [champignon](symptom)
- J'ai [angines](symptom), [champignon](symptom), [fatigue](symptom), [maux de tete](symptom)
- Ma fille souffre de [angines](symptom), [champignon](symptom), [champignon](symptom), [maux de tete](symptom)
- Mon fils souffre de [champignon](symptom), [maux de tete](symptom), [maux de tete](symptom), [angines](symptom)
- Ma femme souffre de [diarrhée](symptom), [fatigue](symptom), [angines](symptom), [fatigue](symptom)
- Ma fille souffre de [angines](symptom), [champignon](symptom), [maux de tete](symptom), [maux de tete](symptom)
- Je souffre de [maux de tete](symptom), [champignon](symptom), [angines](symptom), [diarrhée](symptom)
- Ma femme souffre de [champignon](symptom), [champignon](symptom), [maux de tete](symptom), [diarrhée](symptom)
- Mes symptômes sont [angines](symptom), [champignon](symptom), [fatigue](symptom), [diarrhée](symptom)
- Ma femme souffre de [angines](symptom), [fatigue](symptom), [maux de tete](symptom), [maux de tete](symptom)
- Mon fils souffre de [angines](symptom), [diarrhée](symptom), [diarrhée](symptom), [champignon](symptom)
- J'ai vraiment mal [champignon](symptom), [angines](symptom), [champignon](symptom), [angines](symptom)
- Ma femme souffre de [champignon](symptom), [angines](symptom), [angines](symptom), [champignon](symptom)
- Mon enfant souffre de [diarrhée](symptom), [fatigue](symptom), [angines](symptom), [angines](symptom)
- Ma fille souffre de [maux de tete](symptom), [maux de tete](symptom), [maux de tete](symptom), [angines](symptom)
- Depuis quelques jours, j'ai eu [maux de tete](symptom), [maux de tete](symptom), [diarrhée](symptom), [angines](symptom)
- J'ai [champignon](symptom), [diarrhée](symptom), [maux de tete](symptom), [fatigue](symptom)
- Mon enfant souffre de [angines](symptom), [maux de tete](symptom), [angines](symptom), [maux de tete](symptom)
- Mon fils souffre de [fatigue](symptom), [champignon](symptom), [diarrhée](symptom), [diarrhée](symptom)
- Mon mari souffre de [diarrhée](symptom), [diarrhée](symptom), [maux de tete](symptom), [champignon](symptom)
- Ma femme souffre de [champignon](symptom), [diarrhée](symptom), [fatigue](symptom), [angines](symptom)
- Mes symptômes sont [champignon](symptom), [fatigue](symptom), [angines](symptom), [champignon](symptom)
- J'ai [champignon](symptom), [maux de tete](symptom), [angines](symptom), [maux de tete](symptom)
- Mon mari souffre de [champignon](symptom), [champignon](symptom), [fatigue](symptom), [champignon](symptom)
- Mes symptômes sont [diarrhée](symptom), [champignon](symptom), [maux de tete](symptom), [fatigue](symptom)
- Je ne me sens pas bien, j'ai [fatigue](symptom), [diarrhée](symptom), [maux de tete](symptom), [diarrhée](symptom)
- Mon fils souffre de [maux de tete](symptom), [diarrhée](symptom), [diarrhée](symptom), [fatigue](symptom)
- Ma fille souffre de [maux de tete](symptom), [fatigue](symptom), [maux de tete](symptom), [maux de tete](symptom)
- Mon fils souffre de [champignon](symptom), [champignon](symptom), [maux de tete](symptom), [angines](symptom)
- Mes symptômes sont [angines](symptom), [fatigue](symptom), [fatigue](symptom), [maux de tete](symptom)
- Ma femme souffre de [angines](symptom), [champignon](symptom), [diarrhée](symptom), [diarrhée](symptom)
- Mon fils souffre de [fatigue](symptom), [maux de tete](symptom), [fatigue](symptom), [maux de tete](symptom)
- Mes symptômes sont [fatigue](symptom), [champignon](symptom), [angines](symptom), [angines](symptom)
- Je souffre de [angines](symptom), [angines](symptom), [maux de tete](symptom), [champignon](symptom)
- J'ai vraiment mal [diarrhée](symptom), [champignon](symptom), [angines](symptom), [fatigue](symptom)
- J'ai vraiment mal [champignon](symptom), [fatigue](symptom), [diarrhée](symptom), [diarrhée](symptom)
- Mon fils souffre de [diarrhée](symptom), [angines](symptom), [fatigue](symptom), [maux de tete](symptom)
- Mes symptômes sont [maux de tete](symptom), [champignon](symptom), [champignon](symptom), [diarrhée](symptom)
- J'ai vraiment mal [champignon](symptom), [diarrhée](symptom), [angines](symptom), [maux de tete](symptom)
- J'ai [angines](symptom), [maux de tete](symptom), [maux de tete](symptom), [maux de tete](symptom)
- J'ai vraiment mal [diarrhée](symptom), [angines](symptom), [maux de tete](symptom), [angines](symptom)
- Ma fille souffre de [angines](symptom), [maux de tete](symptom), [diarrhée](symptom), [maux de tete](symptom)

SyntaxError: invalid syntax (1878171043.py, line 1)